# Google's Gemini 2.0 

## Information Extraction: Tabular Data

In [1]:
import os

GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

In [2]:
import pandas as pd

from google import genai
from google.genai import types

from pydantic import BaseModel, Field

In [3]:
# Create a client
client = genai.Client(api_key=GEMINI_API_KEY)

# Specify the model you are going to use
model_id =  "gemini-2.0-flash" # or "gemini-2.0-flash-lite-preview-02-05"  , "gemini-2.0-pro-exp-02-05"

----

#### PDF ...

Since the PDF file in our example is less than 20MB, we can send its contents inline with our request to the model.

In [4]:
os.stat("samples/Press_release_car_registrations_February_2025.pdf")

os.stat_result(st_mode=33188, st_ino=927053, st_dev=2049, st_nlink=1, st_uid=1001, st_gid=1002, st_size=186261, st_atime=1745387131, st_mtime=1744864115, st_ctime=1744864115)

In [5]:
with open("samples/Press_release_car_registrations_February_2025.pdf", "rb") as f:
      file_bytes = f.read()

Feb2025_pdf = types.Part.from_bytes(
    data=file_bytes,
    mime_type='application/pdf',
)

----

### Function for calling the Google Gen AI model

* specifies that the response will be of MIME type `application/json` in the given `response_schema`
* sets the system instructions
* sends the input file along with the user prompt

In [6]:
def extract_structured_data(model_id:str, prompt:str, file_part:types.Part, response_schema:BaseModel):
    """Given the Google Gen AI model,
       a prompt for the model,
       the raw bytes from a PDF file (max. 20MB!) which we send inline with our request,
       and a schema for the structured response we wish to obtain:

       Return a structured response (MIME type application/json) for the given prompt.
    """
    response = client.models.generate_content(
        model=model_id,

        config=types.GenerateContentConfig(
            system_instruction=SYSTEM_LEVEL_INSTRUCTION,
            response_mime_type='application/json', 
            response_schema=response_schema,
            seed=42
        ),
        
        contents=[
            file_part,            
            prompt,
        ]
    )
    
    # Convert the response to the Pydantic model and return it
    #print(response)
    return response.parsed

----

In [7]:
SYSTEM_LEVEL_INSTRUCTION = (
    "You are an expert data analyst, specializing in information extraction from PDFs.  "
    "You especially enjoy parsing out tabular data, always being completely accurate when extracting table parts "
    "such as the row and column headers, and table cells. "
    "You always understand the layout of a table, and know how to return empty values."
).strip()

----

#### NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY, 2025-February

![ACEA Press Release, page 3, NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY, 2025-Feb](samples/Press_release_car_registrations_February_2025_p3.png "NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY, page 3, ACEA Press Release, 2025-Feb")

c.f. NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY on page 3, [samples/Press_release_car_registrations_February_2025.pdf](samples/Press_release_car_registrations_February_2025.pdf)

### Prompt design: NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY

In [8]:
table1_prompt="""
Retrieve the entire contents for the NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE, MONTHLY table in the given PDF.

Parse this table row by row, starting from the top row and going to the bottom row of the table.
Each row represents new car registrations for a country or geopolitical entity.

Parse all 21 columns for each row, starting from left to right.
The 21 columns are grouped into 7 major column categories.

There are 7 major column categories:
- BATTERY ELECTRIC
- PLUG-IN HYBRID
- HYBRID ELECTRIC
- OTHERS
- PETROL
- DIESEL
- TOTAL

Each major column category has 3 values for:
- current month in current year
- current month in previous year
- % change current yy / previous yy

If any of the 3 values in a major column category are missing or empty, represent that value with ??.

Be aware that it is possible for all 3 values in a major column category to be empty!
Example:
|        | BATTERY ELECTRIC | PLUG-IN HYBRID | ...
|Romania | 1,164 217 -65 |   | ...
""".strip()

#### Zero-shot / Minimal Prompt Design

_... will likely not work!_


In [9]:
%%time

response = client.models.generate_content(
        model=model_id,

        config=types.GenerateContentConfig(
            system_instruction=SYSTEM_LEVEL_INSTRUCTION,
            response_mime_type='application/json', 
        ),
        
        contents=[
            Feb2025_pdf,            
            table1_prompt,
        ]
    )

print(response.text)

[
  {
    "BATTERY ELECTRIC": {
      "February 2025": "4,233",
      "February 2024": "3,322",
      "% change 25/24": "+27.4"
    },
    "PLUG-IN HYBRID": {
      "February 2025": "1,613",
      "February 2024": "1,335",
      "% change 25/24": "+20.8"
    },
    "HYBRID ELECTRIC": {
      "February 2025": "5,549",
      "February 2024": "4,691",
      "% change 25/24": "+18.3"
    },
    "OTHERS": {
      "February 2025": "0",
      "February 2024": "0",
      "% change 25/24": "??"
    },
    "PETROL": {
      "February 2025": "5,736",
      "February 2024": "6,527",
      "% change 25/24": "-12.1"
    },
    "DIESEL": {
      "February 2025": "2,488",
      "February 2024": "4,135",
      "% change 25/24": "-39.8"
    },
    "TOTAL": {
      "February 2025": "19,619",
      "February 2024": "20,010",
      "% change 25/24": "-2.0"
    }
  },
  {
    "BATTERY ELECTRIC": {
      "February 2025": "13,040",
      "February 2024": "9,385",
      "% change 25/24": "+38.9"
    },
    "PL

### Structured output: 

#### NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY

_... we need to improve the accuracy of the output for this information retrieval operation... but how?_

![Accuracy optimization, pattern 1: prompt design & structured output](samples/accuracy_optimization_pattern_01.png "Optimizing accuracy in this use case via prompt design and structured output")

In [10]:
class Table1Row(BaseModel):
    name: str = Field(description="name of country or geopolitical entity for this table row")
    
    values: list[str] = Field(description=(
        "list of exactly 21 values, 3 for each the following major column category: BATTERY ELECTRIC, PLUG-IN HYBRID, HYBRID ELECTRIC, OTHERS, PETROL, DIESEL, and TOTAL. if any of the values are blank or missing, represent with an ??"
    ))

class Table1(BaseModel):
    rows: list[Table1Row] = Field(description="List of Table1Row objects that make up the table")

In [11]:
%%time

table_data = extract_structured_data(model_id, table1_prompt, Feb2025_pdf, Table1)

CPU times: user 13.1 ms, sys: 114 μs, total: 13.2 ms
Wall time: 32.8 s


In [12]:
row_headers, data = [], []
for row in table_data.model_dump()['rows']:
    row_headers.append(row['name'])
    data.append(row['values'])

df = pd.DataFrame(data, index=row_headers)
df.replace('??', '', inplace=True)

print(df.shape)
print()
df

(34, 21)



,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
Austria,"4,233","3,322",+27.4,"1,613","1,335",+20.8,"5,549","4,691",+18.3,0,...,,"5,736","6,527",-12.1,"2,488","4,135",-39.8,"19,619","20,010",-2.0
Belgium,"13,040","9,385",+38.9,"3,070","8,385",-63.4,"5,383","4,282",+25.7,267,...,-35.7,"17,280","18,918",-8.7,"1,121","2,337",-52.0,"40,161","43,722",-8.1
Bulgaria,126,122,+3.3,34,31,+9.7,105,73,+43.8,0,...,,"2,781","2,868",-3.0,348,510,-31.8,"3,394","3,604",-5.8
Croatia,53,50,+6.0,140,94,+48.9,"1,629","1,455",+12.0,101,...,-8.2,"1,644","1,898",-13.4,678,923,-26.5,"4,245","4,530",-6.3
Cyprus,107,105,+1.9,78,48,+62.5,579,609,-4.9,0,...,,456,712,-36.0,64,27,+137.0,"1,284","1,501",-14.5
Czechia,737,438,+68.3,557,450,+23.8,"3,634","3,577",+1.6,470,...,-18.8,"8,844","9,723",-9.0,"3,531","3,561",-0.8,"17,773","18,328",-3.0
Denmark,"7,724","4,974",+55.3,312,525,-40.6,"1,453","1,941",-25.1,0,...,,"1,908","3,415",-44.1,220,363,-39.4,"11,617","11,218",+3.6
Estonia,59,89,-33.7,69,59,+16.9,307,609,-49.6,0,...,-100.0,132,430,-69.3,57,244,-76.6,624,"1,440",-56.7
Finland,"1,563","1,330",+17.5,"1,035","1,237",-16.3,"1,398","1,735",-19.4,0,...,-100.0,598,736,-18.8,190,297,-36.0,"4,784","5,374",-11.0
France,25.335,"25,825",-1.9,"6,451","11,732",-45.0,"62,146","41,227",+50.7,"5,821",...,+5.9,"35,110","48,095",-27.0,"6,707","10,221",-34.4,"141,570","142,595",-0.7


In [13]:
df.loc[[
    'Austria',
    'Bulgaria',
    'Cyprus',
    'Denmark',
    'Ireland',
    'Latvia',
    'Luxembourg', 
    'Malta',
    'Romania',    # empty values at indices 3, 4, 5!
    'Iceland', 
    'Norway',
    'United Kingdom'
]].T

,Austria,Bulgaria,Cyprus,Denmark,Ireland,Latvia,Luxembourg,Malta,Romania,Iceland,Norway,United Kingdom
0,"4,233",126,107,"7,724","2,512",80,"1,134",46,724,248,"8,477","21,244"
1,"3,322",122,105,"4,974","1,856",78,942,260,"1,109",97,"6,043","14,991"
2,+27.4,+3.3,+1.9,+55.3,+35.3,+2.6,+20.4,-82.3,-34.7,+155.7,+40.3,+41.7
3,"1,613",34,78,312,"2,100",134,291,27,,104,136,"7,273"
4,"1,335",31,48,525,"1,420",28,385,58,,96,122,"6,098"
5,+20.8,+9.7,+62.5,-40.6,+47.9,+378.6,-24.4,-53.4,,+8.3,+11.5,+19.3
6,"5,549",105,579,"1,453","3,137",0,"1,196",98,"5,510",199,154,"29,849"
7,"4,691",73,609,"1,941","2,940",0,"1,002",124,"3,736",95,259,"26,140"
8,+18.3,+43.8,-4.9,-25.1,+6.7,,+19.4,-21.0,+47.5,+109.5,-40.5,+14.2
9,0,0,0,0,0,18,0,0,"1,354",0,0,0


In [14]:
df.to_csv('samples/acea_1_202502_raw.csv', encoding='CP932', header=False)

----

#### NEW CAR REGISTRATIONS BY MANUFACTURER EU + EFTA + UK, 2025-February

![ACEA Press Release, page 6, NEW CAR REGISTRATIONS BY MANUFACTURER EU + EFTA + UK, 2025-Feb](samples/Press_release_car_registrations_February_2025_p6.png "NEW CAR REGISTRATIONS BY MANUFACTURER EU + EFTA + UK, page 6, ACEA Press Release, 2025-Feb")

c.f. NEW CAR REGISTRATIONS BY MANUFACTURER EU + EFTA + UK on page 6, [samples/Press_release_car_registrations_February_2025.pdf](samples/Press_release_car_registrations_February_2025.pdf)

In [15]:
table2_prompt ="""
Retrieve the entire contents for the NEW CAR REGISTRATIONS BY MANUFACTURER EU + EFTA + UK table on in the given PDF.

Parse this table row by row, starting from the top row and going to the bottom row of the table.
Each row represents new car registrations for a automobile manufacturer or manufacturing group.

Parse all 10 columns for each row, starting from left to right.
The 10 columns are grouped into 2 major column categories.

There are 2 major column categories:
- current month
- JANUARY-current month

For each major column category, you must extract exactly 5 values:
- % share current year
- % share previous year
- Units current year
- Units previous year
- % change current yy / previous yy

If there are any values that are missing or empty, represent that value with ??.
""".strip()

In [16]:
class Table2Row(BaseModel):
    name: str = Field(description="name of automobile manufacturer or manufacturer group for this table row. do not include any any superscript characters")
    
    values: list[str] = Field(description=(
        "list of exactly 10 values, 5 for each the following major column categories: current month, and JANUARY-current month. if any of the values are blank or missing, represent with an ??"
    ))

class Table2(BaseModel):
    rows: list[Table2Row] = Field(description="List of Table2Row objects that make up the table")

In [17]:
%%time

table_data = extract_structured_data(model_id, table2_prompt, Feb2025_pdf, Table2)

CPU times: user 6.11 ms, sys: 3.25 ms, total: 9.36 ms
Wall time: 25.3 s


In [18]:
row_headers, data = [], []
for row in table_data.model_dump()['rows']:
    row_headers.append(row['name'])
    data.append(row['values'])

df = pd.DataFrame(data, index=row_headers)
df.replace('??', '', inplace=True)

print(df.shape)
print()
df

(46, 10)



,0,1,2,3,4,5,6,7,8,9
Volkswagen Group,25.9,25.0,"249,873","248,647",+0.5,26.4,25.0,"516,709","503,487",+2.6
Volkswagen,10.8,9.7,"103,681","96,562",+7.4,11.1,9.6,"216,565","193,379",+12.0
Skoda,5.7,5.9,"55,324","58,854",-6.0,5.8,6.0,"114,314","120,441",-5.1
Audi,4.8,4.6,"46,593","45,927",+1.5,4.9,4.9,"95,822","99,291",-3.5
Cupra,2.1,1.6,"20,397","15,418",+32.3,2.1,1.4,"40,869","28,728",+42.3
Seat,1.8,2.2,"16,998","21,884",-22.3,1.7,2.1,"33,575","41,946",-20.0
Porsche,0.7,1.0,"6,267","9,486",-33.9,0.7,0.9,"14,218","18,524",-23.2
Others,0.1,0.1,613,516,+18.8,0.1,0.1,"1,346","1,178",+14.3
Stellantis,16.2,18.7,"155,970","186,151",-16.2,15.8,18.4,"310,091","369,469",-16.1
Peugeot,5.8,5.8,"56,016","58,094",-3.6,5.7,5.7,"111,443","115,484",-3.5


In [19]:
df.to_csv('samples/acea_2_202502_raw.csv', encoding='CP932', header=False)

<hr width=40%/>

### Check: 2025-Jan PDF



In [20]:
with open("samples/Press_release_car_registrations_January_2025_rev.pdf", "rb") as f:
      file_bytes = f.read()

Jan2025_pdf = types.Part.from_bytes(
    data=file_bytes,
    mime_type='application/pdf',
)

### NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY, 2025-January

![ACEA Press Release, page 3, NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY, 2025-Jan](samples/Press_release_car_registrations_January_2025_p3.png "NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY, page 3, ACEA Press Release, 2025-Jan")

c.f. NEW CAR REGISTRATIONS BY MARKET AND POWER SOURCE MONTHLY on page 3, [samples/Press_release_car_registrations_January_2025_rev.pdf](samples/Press_release_car_registrations_January_2025_rev.pdf)

In [21]:
%%time

table_data = extract_structured_data(model_id, table1_prompt, Jan2025_pdf, Table1)

CPU times: user 13.2 ms, sys: 137 μs, total: 13.4 ms
Wall time: 35.8 s


In [22]:
row_headers, data = [], []
for row in table_data.model_dump()['rows']:
    row_headers.append(row['name'])
    data.append(row['values'])

df = pd.DataFrame(data, index=row_headers)
df.replace('??', '', inplace=True)

print(df.shape)
print()
df

(34, 21)



,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
Austria,"3,822","2,823",+35.4,"1,642","1,469",+11.8,"5,405","4,069",+32.8,0,...,-100.0,"6,505","5,681",+14.5,"3,074","3,506",-12.3,"20,448","17,552",+16.5
Belgium,"13,712","9,995",+37.2,"3,996","11,972",-66.6,"4,458","4,170",+6.9,293,...,+10.2,"16,600","17,461",-4.9,"1,535","2,803",-45.2,"40,594","46,667",-13.0
Bulgaria,142,162,-12.3,38,49,-22.4,78,61,+27.9,0,...,,"2,228","3,270",-31.9,294,435,-32.4,"2,780","3,977",-30.1
Croatia,69,82,-15.9,152,119,+27.7,"1,556","1,216",+28.0,142,...,+11.8,"1,977","2,045",-3.3,576,903,-36.2,"4,472","4,492",-0.4
Cyprus,149,65,+129.2,93,68,+36.8,689,679,+1.5,0,...,,609,630,-3.3,25,85,-70.6,"1,565","1,527",+2.5
Czechia,981,473,+107.4,641,431,+48.7,"4,225","3,386",+24.8,378,...,-34.9,"9,362","11,167",-16.2,"3,760","4,323",-13.0,"19,347","20,361",-5.0
Denmark,"6,961","3,117",+123.3,190,366,-48.1,"1,377","1,746",-21.1,0,...,,"1,955","3,277",-40.3,351,333,+5.4,"10,834","8,839",+22.6
Estonia,83,136,-39.0,77,55,+40.0,254,683,-62.8,0,...,-100.0,82,413,-80.1,35,281,-87.5,531,"1,577",-66.3
Finland,"1,639","1,587",+3.3,"1,366","1,545",-11.6,"1,776","2,284",-22.2,0,...,-100.0,706,801,-11.9,255,354,-28.0,"5,742","6,590",-12.9
France,"19,923","20,017",-0.5,"4,852","10,549",-54.0,"51,447","33,810",+52.2,"3,521",...,-46.3,"29,974","41,728",-28.2,"4,956","9,620",-48.5,"114,673","122,285",-6.2


In [23]:
df.loc[[
    'Bulgaria',
    'Cyprus',
    'Denmark',
    'Ireland',
    'Latvia',
    'Luxembourg', 
    'Malta',
    'Romania', 
    'Iceland', 
    'Norway',
    'United Kingdom'
]].T

,Bulgaria,Cyprus,Denmark,Ireland,Latvia,Luxembourg,Malta,Romania,Iceland,Norway,United Kingdom
0,142,149,"6,961","4,923",84,"1,167",211,"1,164",226,"8,954","29,634"
1,162,65,"3,117","4,093",86,779,253,"1,631",169,"4,717","20,935"
2,-12.3,+129.2,+123.3,+20.3,-2.3,+49.8,-16.6,-28.6,+33.7,+89.8,+41.6
3,38,93,190,"4,915",126,319,44,0,186,95,"12,598"
4,49,68,366,"2,999",32,396,61,0,90,94,"11,944"
5,-22.4,+36.8,-48.1,+63.9,+293.8,-19.4,-27.9,,+106.7,+1.1,+5.5
6,78,689,"1,377","8,725",0,957,134,"5,284",61,124,"51,785"
7,61,679,"1,746","7,761",0,847,118,"4,215",72,164,"47,435"
8,+27.9,+1.5,-21.1,+12.4,,+13.0,+13.6,+25.4,-15.3,-24.4,+9.2
9,0,0,0,0,36,0,0,"1,511",0,0,0


<hr width=40%/>

#### NEW CAR REGISTRATIONS BY MANUFACTURER EU + EFTA + UK, 2025-January

![ACEA Press Release, page 6, NEW CAR REGISTRATIONS BY MANUFACTURER EU + EFTA + UK, 2025-Jan](samples/Press_release_car_registrations_January_2025_p6.png "NEW CAR REGISTRATIONS BY MANUFACTURER EU + EFTA + UK, page 6, ACEA Press Release, 2025-Jan")

c.f. NEW CAR REGISTRATIONS BY MANUFACTURER EU + EFTA + UK on page 6, [samples/Press_release_car_registrations_January_2025_rev.pdf](samples/Press_release_car_registrations_January_2025_rev.pdf)

In [24]:
%%time

table_data = extract_structured_data(model_id, table2_prompt, Jan2025_pdf, Table2)

CPU times: user 10.5 ms, sys: 820 μs, total: 11.3 ms
Wall time: 26.8 s


In [25]:
row_headers, data = [], []
for row in table_data.model_dump()['rows']:
    row_headers.append(row['name'])
    data.append(row['values'])

df = pd.DataFrame(data, index=row_headers)
df.replace('??', '', inplace=True)

print(df.shape)
print()
df

(46, 10)



,0,1,2,3,4,5,6,7,8,9
Volkswagen Group,27.0,25.1,"268,409","254,840",+5.3,27.0,25.1,"268,409","254,840",+5.3
Volkswagen,11.3,9.5,"112,844","96,817",+16.6,11.3,9.5,"112,844","96,817",+16.6
Skoda,6.0,6.1,"59,465","61,587",-3.4,6.0,6.1,"59,465","61,587",-3.4
Audi,4.9,5.2,"49,220","53,364",-7.8,4.9,5.2,"49,220","53,364",-7.8
Cupra,2.1,1.3,"20,855","13,310",+56.7,2.1,1.3,"20,855","13,310",+56.7
Seat,1.7,2.0,"17,161","20,062",-14.5,1.7,2.0,"17,161","20,062",-14.5
Porsche,0.8,0.9,"8,120","9,038",-10.2,0.8,0.9,"8,120","9,038",-10.2
Others,0.1,0.1,744,662,+12.4,0.1,0.1,744,662,+12.4
Stellantis,15.5,18.0,"154,079","183,319",-16.0,15.5,18.0,"154,079","183,319",-16.0
Peugeot,5.6,5.6,"55,445","57,390",-3.4,5.6,5.6,"55,445","57,390",-3.4


----

In [26]:
# housekeeping...
for f in client.files.list():
    client.files.delete(name=f.name)